In [ ]:
import pandas as pd
import numpy as np
import ast


# Read the CSV file
df = pd.read_csv('../datasets/tripdata_2022_05_05.csv')
df.head()

In [4]:
# Drop unwanted columns
columns_to_drop = [
    'fecha',
    'idBike',
    'fleet',
    'locktype',
    'unlocktype',
    'address_lock',
    'address_unlock',
    'dock_unlock',
    'dock_lock'
]

df_cleaned = df.drop(columns=columns_to_drop)

# Optional: Preview the result
df_cleaned.head()
df_cleaned.to_csv('../datasets/tripdata_2022_05_05.csv', index=False)

In [ ]:
#Creating all_stations.csv
def extract_coordinates(geo_str):
    try:
        geo_dict = ast.literal_eval(geo_str)
        lon, lat = geo_dict['coordinates']
        return pd.Series([lat, lon])
    except:
        return pd.Series([None, None])

# Extract lat/lon
df[['lat', 'lon']] = df['geolocation_unlock'].apply(extract_coordinates)

# Select station ID, name, and coordinates
station_df = df[['station_unlock', 'unlock_station_name', 'lat', 'lon']]

# Drop duplicates so we only have one row per station
station_df = station_df.drop_duplicates()

# Save to CSV
station_df.to_csv('../datasets/all_stations.csv', index=False)
station_df.head()

In [ ]:
# Creating all_trips.csv
trips_df = df[['id', 'unlock_date', 'lock_date', 'station_unlock', 'station_lock', 'trip_minutes']]

# Remove rows with missing critical data (e.g., trips with no end time or duration)
trips_df = trips_df.dropna(subset=['unlock_date', 'lock_date', 'trip_minutes'])

# Save to CSV
trips_df.to_csv('../datasets/all_trips.csv', index=False)
trips_df.head()